# Assignment 7

## Submit as an HTML file

### Print your name below

In [1]:
print("Joshua Han")

Joshua Han


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [1]:
# Write your answer here:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf


#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [29]:
results = pd.read_csv("data/results.csv")

print(results.iloc[0:5,:])

   resultId  raceId  driverId  constructorId number  grid position  \
0         1      18         1              1     22     1        1   
1         2      18         2              2      3     5        2   
2         3      18         3              3      7     7        3   
3         4      18         4              4      5    11        4   
4         5      18         5              1     23     3        5   

  positionText  positionOrder  points  laps         time milliseconds  \
0            1              1    10.0    58  1:34:50.616      5690616   
1            2              2     8.0    58       +5.478      5696094   
2            3              3     6.0    58       +8.163      5698779   
3            4              4     5.0    58      +17.181      5707797   
4            5              5     4.0    58      +18.014      5708630   

  fastestLap rank fastestLapTime fastestLapSpeed  statusId  
0         39    2       1:27.452         218.300         1  
1         41    3 

### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [30]:
# Write your answer here
print(results.dtypes)

list_old = ['\\N']
list_new = [np.nan]
results["milliseconds"] = results["milliseconds"].replace(list_old, list_new)
results["race_time_ms"] = pd.to_numeric(results["milliseconds"])

resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
dtype: object


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [21]:
# Write your answer here
bins_x = [0,3,10,20,float("inf")]
labels_x = ["Podium", "Points", "Midfield", "Backmarker"]

results["finish_category"] = pd.cut(results["positionOrder"],
                                    bins = bins_x,
                                    right = True,
                                    labels = labels_x)



### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [23]:
# Write your answer here
results["race_duration_minutes"] = results["race_time_ms"] / (1000*60)

avg_race_duration = (results.groupby("constructorId")
                     .agg(mean_duration = ('race_duration_minutes','mean'))
                     .sort_values('mean_duration', ascending=True))

print(avg_race_duration.iloc[0:5,:])

               mean_duration
constructorId               
35                 76.710777
29                 77.604125
41                 87.046767
16                 89.428828
53                 89.658852


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [25]:
# Write your answer here
results_agg = (results.groupby("driverId")
               .agg(mean_finish = ('positionOrder','mean'),
                    total_points = ('points','sum'),
                    num_races = ('positionOrder','count'),
                    best_finish = ('positionOrder','min'))
               .sort_values('total_points', ascending = False))

print(results_agg.iloc[0:10,:])


          mean_finish  total_points  num_races  best_finish
driverId                                                   
1            4.787097        4396.5        310            1
20           7.093333        3098.0        300            1
4            8.494413        2061.0        358            1
830          6.533742        1983.5        163            1
8            8.491477        1873.0        352            1
822          7.601990        1778.0        201            1
3            8.252427        1594.5        206            1
30           6.879870        1566.0        308            1
817          9.883621        1307.0        232            1
18           9.695793        1235.0        309            1


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [27]:
# Write your answer here
results = results[["points","grid","laps"]].dropna()

model = smf.ols(formula = 'points ~ grid + laps', data = results)
regression = model.fit()

print(regression.summary())

predicted = regression.predict({'grid': [3], 'laps': [55]})
print(predicted)


                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        19:51:42   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0